In [3]:
"""
Extract Games

Use nhl API https://github.com/dword4/nhlapi to extract games & results

Save results in Google Big Query
"""

credential_keys = '/Users/antoinetl/Documents/code/Google Keys/My First Project-4938b2ab0dc6.json'

In [4]:
import pandas as pd
import requests  
from pandas.io.json import json_normalize
import datetime

pd.set_option('display.max_columns', None)

# Connexion GBQ
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    credential_keys,
    )
pandas_gbq.context.credentials = credentials
# https://pandas-gbq.readthedocs.io/en/latest/intro.html

In [5]:
def extract_gamestats(gameID):
    
    '''
    Takes the gameID as an input and returns a dataframe that contains
    the teamSkaterStats info for the home/away teams for that specific game.
    '''
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(gameID))
    data = r.json()
    df = pd.json_normalize(data = data['teams'])
    df = df.filter(regex='teamSkaterStats')
    df['gameID'] = gameID
    
    return df

In [6]:
# select date for extraction (july will be a complete season)
start_date = datetime.date(2008, 7, 1)
end_date = datetime.date(2018, 7, 1)

delta = datetime.timedelta(days=1)

data_list = []

while start_date <= end_date:
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/schedule?date=' + start_date.strftime("%Y-%m-%d"))
    data = r.json()
    
    df = pd.json_normalize(data = data['dates'], record_path='games', meta=['date'])
    
    # On ajoute des statistiques au niveau des matchs
    # Le if sert a skipper les journees sans matchs
    if df.empty==False :
        pd_list = []
        for games in df['gamePk']:
            tmp = extract_gamestats(gameID=games)
            pd_list.append(tmp)

        pd_tmp = pd.concat(pd_list, sort='False', ignore_index=True)
        df = df.merge(pd_tmp, left_on='gamePk', right_on='gameID', how='left')

        data_list.append(df)
        
        start_date += delta
    else:
        start_date += delta
    
    if (start_date.day == 1):
        print(start_date)

2008-08-01
2008-09-01
2008-10-01
2008-11-01
2008-12-01
2009-01-01
2009-02-01
2009-03-01
2009-04-01
2009-05-01
2009-06-01
2009-07-01
2009-08-01
2009-09-01
2009-10-01
2009-11-01
2009-12-01
2010-01-01
2010-02-01
2010-03-01
2010-04-01
2010-05-01
2010-06-01
2010-07-01
2010-08-01
2010-09-01
2010-10-01
2010-11-01
2010-12-01
2011-01-01
2011-02-01
2011-03-01
2011-04-01
2011-05-01
2011-06-01
2011-07-01
2011-08-01
2011-09-01
2011-10-01
2011-11-01
2011-12-01
2012-01-01
2012-02-01
2012-03-01
2012-04-01
2012-05-01
2012-06-01
2012-07-01
2012-08-01
2012-09-01
2012-10-01
2012-11-01
2012-12-01
2013-01-01
2013-02-01
2013-03-01
2013-04-01
2013-05-01
2013-06-01
2013-07-01
2013-08-01
2013-09-01
2013-10-01
2013-11-01
2013-12-01
2014-01-01
2014-02-01
2014-03-01
2014-04-01
2014-05-01
2014-06-01
2014-07-01
2014-08-01
2014-09-01
2014-10-01
2014-11-01
2014-12-01
2015-01-01
2015-02-01
2015-03-01
2015-04-01
2015-05-01
2015-06-01
2015-07-01
2015-08-01
2015-09-01
2015-10-01
2015-11-01
2015-12-01
2016-01-01
2016-02-01

KeyError: 'dates'

In [8]:
# Pour stack la version
df = pd.concat(data_list, sort='False', ignore_index=True)

In [10]:
# replace columns in dataframe because GCP does not support '.' in column indices
df.columns = df.columns.str.replace(r".", "_")

In [11]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "rational-world-288611"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'My_dataset.schedule_and_results'

pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists='replace')

1it [00:07,  7.92s/it]


,away.teamStats.teamSkaterStats.blocked,away.teamStats.teamSkaterStats.giveaways,away.teamStats.teamSkaterStats.goals,away.teamStats.teamSkaterStats.hits,away.teamStats.teamSkaterStats.pim,away.teamStats.teamSkaterStats.powerPlayGoals,away.teamStats.teamSkaterStats.powerPlayOpportunities,away.teamStats.teamSkaterStats.shots,away.teamStats.teamSkaterStats.takeaways,gameID,gamePk,home.teamStats.teamSkaterStats.blocked,home.teamStats.teamSkaterStats.giveaways,home.teamStats.teamSkaterStats.goals,home.teamStats.teamSkaterStats.hits,home.teamStats.teamSkaterStats.pim,home.teamStats.teamSkaterStats.powerPlayGoals,home.teamStats.teamSkaterStats.powerPlayOpportunities,home.teamStats.teamSkaterStats.shots,home.teamStats.teamSkaterStats.takeaways,teams.away.leagueRecord.losses,teams.away.leagueRecord.ot,teams.away.leagueRecord.wins,teams.away.score,teams.away.team.id,teams.home.leagueRecord.losses,teams.home.leagueRecord.ot,teams.home.leagueRecord.wins,teams.home.score,teams.home.team.id,venue.id
count,10796.000000,10796.000000,10796.000000,10796.000000,10796.000000,10796.000000,10796.000000,10639.000000,10796.000000,1.079600e+04,1.079600e+04,10796.000000,10796.000000,10796.000000,10796.00000,10796.000000,10796.000000,10796.000000,10639.000000,10796.000000,10796.000000,10051.000000,10796.000000,10796.000000,10796.000000,10796.000000,10051.000000,10796.000000,10796.000000,10796.000000,6051.000000
mean,13.861801,6.486847,2.517877,21.719248,11.741849,0.562616,3.207021,28.640380,5.546684,2.011490e+09,2.011490e+09,13.252408,9.148944,2.791775,23.68442,11.092627,0.659596,3.498055,30.483504,7.455539,13.444794,4.447518,17.669044,2.636995,16.493423,13.410152,4.443836,17.694146,2.913116,29.611060,5055.502727
std,5.720702,4.316638,1.604924,9.373166,9.109389,0.750771,1.715318,7.339369,3.413911,2.353985e+06,2.353985e+06,5.627442,5.131657,1.694224,9.56981,8.870962,0.811110,1.813514,7.653873,4.209494,10.356490,3.573661,13.283702,1.590627,10.648602,10.361273,3.582865,13.262589,1.661054,273.046205,141.431594
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.008010e+09,2.008010e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000
25%,10.000000,3.000000,1.000000,16.000000,6.000000,0.000000,2.000000,24.000000,3.000000,2.009021e+09,2.009021e+09,10.000000,6.000000,2.000000,18.00000,6.000000,0.000000,2.000000,26.000000,5.000000,4.000000,1.000000,6.000000,1.000000,8.000000,4.000000,1.000000,6.000000,2.000000,8.000000,5043.000000
50%,14.000000,6.000000,2.000000,21.000000,10.000000,0.000000,3.000000,28.000000,5.000000,2.011021e+09,2.011021e+09,13.000000,9.000000,3.000000,23.00000,9.000000,0.000000,3.000000,30.000000,7.000000,12.000000,4.000000,16.000000,3.000000,16.000000,12.000000,4.000000,16.000000,3.000000,16.000000,5064.000000
75%,17.000000,9.000000,4.000000,27.000000,15.000000,1.000000,4.000000,33.000000,8.000000,2.014020e+09,2.014020e+09,17.000000,12.000000,4.000000,29.00000,14.000000,1.000000,5.000000,35.000000,10.000000,21.000000,7.000000,28.000000,4.000000,24.000000,21.000000,7.000000,28.000000,4.000000,24.000000,5081.000000
max,46.000000,31.000000,17.000000,69.000000,163.000000,5.000000,11.000000,59.000000,27.000000,2.015041e+09,2.015041e+09,40.000000,36.000000,12.000000,71.00000,183.000000,6.000000,12.000000,63.000000,31.000000,51.000000,18.000000,56.000000,17.000000,98.000000,51.000000,18.000000,56.000000,12.000000,7256.000000,5260.000000
